# Задание 1

Аудио длинной в 1 секунду и частотой дискретизации в 16 кГЦ загоняем в SIFT с окном в 25 мс и шагом в 10 мс.\
Требуется найти количество поместившихся в него временных окон

In [25]:
def count_stft_frames(sample_rate, sample_duration, window_size, step_size):
    # переводим из секунд в количество семплов
    # длинна в секундах на частоту дискретизации в герцах(семплов в секунду)
    window_size = window_size * sample_rate
    step_size = step_size * sample_rate

    samples = sample_duration * sample_rate  # кличество семплов в аудио

    # считаем количество окон (сколько целых шагов нужно чтобы дойти до последнего + 1 последний)
    frames = (samples - window_size) // step_size + 1  # -> получем только окна которые полностью помещаются в сигнал

    return int(frames)  # frames уже целое из за целочисленного деления

In [26]:
sample_rate = 16000  # частота дискретизации в герцах
sample_duration = 1  # длительность сигнала в секундах
window_size = 0.025  # длина окна в секундах
step_size = 0.010  # шаг окна в секундах

In [27]:
frames = count_stft_frames(sample_rate, sample_duration, window_size, step_size)

print(f"Временных окон: {frames}")

Временных окон: 98


# Задание 2

Модель: y_pred = wx + b\
Функция потерь: L = (y_pred - y_true)^2

вес w = 2.0 и смещение b = 1.0

на входе x = 3\
ожидаемый предикт y_true = 10

Требуется посчитать градиенты для w и b

In [33]:
# модель y_pred = wx + b
class Model:
    def __init__(self, w, b):
        self.w = w
        self.b = b

    def forward(self, x):  # вычисление линейного выражения (предикта)
        y = self.w * x + self.b
        return y

    def backward(self, x, y_true):
        y_pred = self.forward(x)

        # производная по x от (wx+b - y_true)^2 = 2*(wx+b - y_true) * (производная по w от (wx+b - y_true) которая равна x)
        dL_dw = 2 * (y_pred - y_true) * x

        # производная по x от (wx+b - y_true)^2 = 2*(wx+b - y_true) * (производная по b от (wx+b - y_true) которая равна 1)
        dL_db = 2 * (y_pred - y_true)

        return dL_dw, dL_db

In [34]:
# лосс L = (y_pred - y_true)^2
def loss(model, x, y):
    L = (model.forward(x) - y) ** 2

    return L

In [35]:
x = 3
y_true = 10

In [120]:
model = Model(2.0, 1.0)

In [121]:
# вычислим лосс для предикта модели
L = loss(model, x, y_true)
print(L)

9.0


In [122]:
# найдем частные производные функции потерь по обучаемым параметрам
# dL по dw и dL по db
dL_dw, dL_db = model.backward(x, y_true)
print("dL по dw:", dL_dw)
print("dL по db:", dL_db)

dL по dw: -18.0
dL по db: -6.0


# Задание 3

Собираемся обучить модель для решения задачи распознавания речи.



In [ ]:
# ASR Data Preparation Notebook
# Автор: ChatGPT
# Задача: подготовка данных для обучения модели ASR

import os
import json
import librosa
import soundfile as sf
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pathlib import Path
from IPython.display import Audio, display
import random

# Параметры
AUDIO_FILE = "audio.ogg"  # замените на путь к вашему аудио файлу
TRANSCRIPT_FILE = "transcript.json"  # замените на путь к json-файлу с транскрипцией
OUTPUT_DIR = "chunks"
MAX_DURATION = 30.0  # сек
SAMPLE_RATE = 16000
N_MELS = 80

# Создание выходной директории
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Загрузка и конвертация аудио
print("\n[1] Загрузка и ресемплирование аудио")
y, sr = librosa.load(AUDIO_FILE, sr=SAMPLE_RATE)
print(f"Аудио загружено: {len(y)/sr:.2f} сек, частота дискретизации: {sr} Гц")

# Загрузка транскрипции
print("\n[2] Загрузка транскрипции")
with open(TRANSCRIPT_FILE, 'r', encoding='utf-8') as f:
    transcript = json.load(f)

# Парсинг фраз и разбиение по сегментам <= 30 сек
print("\n[3] Разделение на сегменты")
chunks = []
temp_chunk = []
temp_duration = 0.0

for phrase in transcript:
    for term in phrase['terms']:
        start = term['start']
        end = term['end']
        duration = end - start
        if temp_duration + duration <= MAX_DURATION:
            temp_chunk.append(term)
            temp_duration += duration
        else:
            chunks.append(temp_chunk)
            temp_chunk = [term]
            temp_duration = duration
if temp_chunk:
    chunks.append(temp_chunk)

print(f"Получено сегментов: {len(chunks)}")

# Сохранение аудио и текста для каждого сегмента
print("\n[4] Сохранение сегментов")
metadata = []

for i, chunk in enumerate(chunks):
    seg_start = chunk[0]['start']
    seg_end = chunk[-1]['end']
    seg_audio = y[int(seg_start * sr):int(seg_end * sr)]
    seg_text = " ".join([t['text_normalized'] for t in chunk])
    file_wav = f"{OUTPUT_DIR}/chunk_{i:03}.wav"
    sf.write(file_wav, seg_audio, sr)
    metadata.append({"audio": file_wav, "text": seg_text})

with open(f"{OUTPUT_DIR}/metadata.json", 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print(f"Сегменты и метаданные сохранены в папку '{OUTPUT_DIR}'")

# Пример визуализации одного из сегментов
print("\n[5] Пример визуализации")
plt.figure(figsize=(10, 3))
plt.title("Waveform одного из сегментов")
plt.plot(np.linspace(0, len(seg_audio) / sr, num=len(seg_audio)), seg_audio)
plt.xlabel("Время (сек)")
plt.ylabel("Амплитуда")
plt.grid(True)
plt.show()

display(Audio(seg_audio, rate=sr))

# [6] Аугментация: pitch shift, добавление шума
print("\n[6] Применение аугментации к аудио")
def add_noise(data, noise_level=0.005):
    noise = np.random.randn(len(data))
    augmented = data + noise_level * noise
    return augmented.astype(np.float32)

def pitch_shift(data, sr, n_steps=2):
    return librosa.effects.pitch_shift(data, sr, n_steps=n_steps)

augmented_dir = f"{OUTPUT_DIR}/augmented"
os.makedirs(augmented_dir, exist_ok=True)

for i, item in enumerate(metadata):
    y_seg, _ = librosa.load(item["audio"], sr=sr)
    # Добавим шум
    y_noise = add_noise(y_seg)
    sf.write(f"{augmented_dir}/chunk_{i:03}_noise.wav", y_noise, sr)
    # Сдвиг тона
    y_pitch = pitch_shift(y_seg, sr, n_steps=random.choice([-2, -1, 1, 2]))
    sf.write(f"{augmented_dir}/chunk_{i:03}_pitch.wav", y_pitch, sr)

print("Аугментированные данные сохранены")

# [7] Преобразование в mel-спектрограммы (для Whisper / QuartzNet / Conformer)
print("\n[7] Генерация mel-спектрограмм")
mel_dir = f"{OUTPUT_DIR}/mel_specs"
os.makedirs(mel_dir, exist_ok=True)

for i, item in enumerate(metadata):
    y_seg, _ = librosa.load(item["audio"], sr=sr)
    mel = librosa.feature.melspectrogram(y=y_seg, sr=sr, n_mels=N_MELS)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    np.save(f"{mel_dir}/chunk_{i:03}.npy", mel_db)

print("Mel-спектрограммы сохранены в numpy-формате")


# Задание 4

LSTM с input_size = 128 и hidden_size = 256

Сколько для одной LSTM ячейки обучаемых параметров

In [ ]:
"""
LSTM ячейка состоит из 4 гейтов каждый из которых по сути яввляется линейным слоем.

Каждый гейт имеет одну и ту же формулу sigmoid(Wf * Xt + Uf * Ht-1 + Bf)
    здесь Xt размера input_size и Ht-1 размера hidden_size
    следовательно Wf размера hidden_size * input_size, Uf размера hidden_size * hidden_size, Bf размера hidden_size

получаем hidden_size * input_size + hidden_size * hidden_size + hidden_size = hidden_size * (input_size + hidden_size + 1) = 98560
98560 * 4 = 394240 параметров суммаро для 4 гейтов в одной LSTM ячейке
----------------------------------------------------------------------

в torch.nn.LSTM реализованно два биаса для каждого гейта. один для входного слоя другой для скрытого что дает
hidden_size * (input_size + hidden_size + 2) = 98816
98816 * 4 = 395264 параметров суммаро для 4 гейтов в одной torch.nn.LSTM ячейке
-------------------------------------------------------------------------------
"""

In [130]:
import torch.nn as nn

input_size = 128
hidden_size = 256
lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)  # создаем LSTM блок

In [131]:
# Считаем количество параметров
total_params = sum(p.numel() for p in lstm.parameters() if p.requires_grad)  # сумма параметров
print(f"Обучаемых параметров:", total_params)

Обучаемых параметров: 395264


# Задание 5

Решаем задачу бинарной классификации аудиозаписей

Имеется 100 записей:
* 1 из класса C1
* 99 из класса C0

Обучен классификатор:
* True Positive (Предсказали С1 при истинном С1) с вероятностью 0.9
* False Positive (Предсказали С1 при истинном С0) с вероятностью 0.03

Найти:
1. Какова вероятность что случайную запись из датасета классификатор отнесет к C1
2. Если случайная запись классифицирована как C1 какова вероятность что она действительно C1

In [ ]:
"""
Заметим что вероятность получить класс C1 равна P(C1) = 0.01 а вероятность класса C0 P(C0) = 0.99
Заметим что события С1 истинное и С0 ложное формируют полную группу гипотез

1. По формуле полной вероятности
    P(C1_predicted) = P(C1_predicted | C1_истинная) * P(C1_истинная) + P(C1_predicted | C0_истинная) * P(C0_истинная)
    P(C1_predicted) = 0.9 * 0.01 + 0.03 * 0.99 = 0.0387 ~= 0.038
                                                 --------------

2. По теореме Байеса
    P(C1_истинная | C1_predicted) =   P(C1_predicted | C1_истинная) * P(C1_истинная)                                                   =
                                     _________________________________________________________________________________________________
                                      P(C1_predicted | C1_истинная) * P(C1_истинная) + P(C1_predicted | C0_истинная) * P(C0_истинная)

     = 0.9 * 0.01 / ( 0.9 * 0.01 + 0.03 * 0.99 ) = 0.2325581395 ~= 0.232
                                                   --------------------

Итого:
1. Вероятность того что случайную запись из датасета классификатор отнесет к C1 ~= 0.038
2. Вероятность того что случайная запись классифицирована как C1 и действительно является C1 ~= 0.232
"""

# Задание 6

Для X1 .. Xn из множества вещественных чисел.\
Необходимо написать аналитическое решение для поиска значения b для минимизации sum( (Xi - b)^2 )


In [ ]:
"""
Хотим минимизировать функцию потерь. Для вектора X найти некое b чтобы сумма квадратов разностей стала минимальной
sum( (Xi - b)^2 )

    Для этого воспользуемся градиентом приравненым к нулю т.е. точкой экстремума.
    Т.к функция гладкая и выпуклая (парабола с ветками вверх) следовательно она имеет только одну точку экстремума являющуюся минимумом.
    Т.е

1. Раскрываем скобки под суммой
sum( Xi^2 - 2 * Xi * b + b^2 )

2. Раскроем по линейности суммы
sum(Xi^2) - 2 * b * sum(Xi) + N * b^2

3. найдем частную производную по b
-2 * sum(Xi) + N * 2b

4. Приравняем производную к нулю для поиска точки экстремума
-2 * sum(Xi) + N * 2b = 0
N * 2b = 2 * sum(Xi)
2 * b = 2 * 1/N * sum(Xi)
b = 1/N * sum(Xi)

Ответ: (оказалось что выгоднее всего с точки зрения минимизации функции потерь выбирать среднее значение для b)
b = 1/N * sum(Xi)
---------------------------------------------------------------------------------------------------------------
"""

# Задание 7

Образуют ли векторы:
* x1 = [1, 0, 1]^T
* x2 = [1, 1, 0]^T
* x3 = [1, 1, 1]^T

Базис векторного пространства в R^3. Написать аналитическое решение

In [ ]:
"""
Проверим векторы на линейную зависимость.
Если окажется что один из векторов представим как линейная комбинация двух других то вектора линейно зависимы -> не образуют базис в R^3

1. Перепишем вектры как линейную комбинацию и приравняем к нулю:
a * [1, 0, 1]^T + b * [1, 1, 0]^T + c * [1, 1, 1]^T = 0

    тогда получится что любой вектор представим как линейная комбинация двух других со знаком минус например:
    a * [1, 0, 1]^T + b * [1, 1, 0]^T = -c * [1, 1, 1]^T

2. Составим и решим систему уравнений
    [1]       [1]       [1]   [0]
a * [0] + b * [1] + c * [1] = [0]
    [1]       [0]       [1]   [0]

 /  a + b + c = 0
<   b + c = 0
 \  a + c = 0

    из второй и третей строки получим:
    b = -c
    a = -c
    подставим в первое:
    -с -с + с= 0
    т.е. -с = 0
    Тогда единственное решение системы будет в точке [0, 0, 0]^T (a=b=c=0)

    следовательно Векторы можно представить как линейную комбинацию друг друга только если все они нулевые.
                  Значит они линейно независимы

А это значит что векторы образуют базис векторного пространства в R^3
----------------------------------------------------------------------
"""